# Машинное обучение, ФКН ВШЭ

## Практическое задание 7. Градиентный бустинг ~~своими руками~~

### Общая информация
Дата выдачи: 07.12.2018

Мягкий дедлайн: 05:59MSK 18.12.2018

Жесткий дедлайн: 05:59MSK 19.12.2018

### Оценивание и штрафы
Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимально допустимая оценка за работу — 10 баллов.

Сдавать задание после указанного срока сдачи нельзя. При выставлении неполного балла за задание в связи с наличием ошибок на усмотрение проверяющего предусмотрена возможность исправить работу на указанных в ответном письме условиях.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов (подробнее о плагиате см. на странице курса). Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

Неэффективная реализация кода может негативно отразиться на оценке.

### Формат сдачи
Задания сдаются через систему anytask. Посылка должна содержать:
* Ноутбук homework-practice-07-Username.ipynb

Username — ваша фамилия и имя на латинице именно в таком порядке

In [ ]:
import pandas as pd
import numpy as np
import catboost as cb
import lightgbm as lgb
import time

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import average_precision_score

__Задание 1. (0.5 балла)__

Мы будем использовать данные соревнования [Home Credit Default Risk](https://www.kaggle.com/c/home-credit-default-risk/data). 
* Загрузите таблицу application_train.csv;
* Запишите в Y столбец с целевой переменной;
* Удалите ненужные столбцы (для этого воспользуйтесь описанием);
* Определите тип столбцов и заполните пропуски - стратегия произвольная;
* Разбейте выборку в соотношении 70:30 с random_state=0.

Так как в данных значительный дисбаланс классов, в качестве метрики качества везде будем использовать площадь под precision-recall кривой.

In [ ]:
X = pd.read_csv('../input/application_train.csv')

In [ ]:
y = X['TARGET']
X = X.drop(['TARGET'], axis=1)

In [ ]:
print(X.columns)
X = X.fillna(0)
categorical_features = []
numerical_features = []
for column in X.columns:
    if (X[column].dtype == 'O'):
        categorical_features = categorical_features + [column]
    else:
        numerical_features = numerical_features + [column]
print(len(categorical_features))        

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7, random_state = 0)

__Задание 2. (1.5 балла)__

Обучите реализации градиентного бустинга LightGBM и Catboost на вещественных признаках без подбора параметров. 
Почему получилась заметная разница в качестве? 

В этом и последующих экспериментах необходимо измерять время обучения моделей.

In [ ]:
st_time = time.time()
model = cb.CatBoostClassifier(task_type = "GPU")
model.fit(X_train[numerical_features], y_train)
print('CatBoost:')
print('time:', time.time() - st_time)
print(average_precision_score(y_test, model.predict_proba(X_test[numerical_features])[:, 1]))

In [ ]:
st_time = time.time()
model = lgb.LGBMClassifier()
model.fit(X_train[numerical_features], y_train)
print('Lgb:')
print('time:', time.time() - st_time)
print(average_precision_score(y_test, model.predict_proba(X_test[numerical_features])[:, 1]))

Ответ: Разница в качестве тут, поскольку CB дольше обучается в этой ситуации, и позволяет лучше обучиться и показать лучший результат

__Задание 3. (2 балла)__

Подберите с CV=3 оптимальные параметры алгоритмов, изменяя:

* глубину деревьев;
* количество деревьев;
* темп обучения;
* оптимизируемый функционал.

Проанализируйте соотношения глубины и количества деревьев в зависимости от алгоритма.

In [ ]:
#st_time = time.time()
#cat_boost_params = {
#    'depth': [2,5,8],
#    'n_estimators':[200, 500, 1000],
#    'learning_rate': [0.03, 0.05, 0.1],
#    'loss_function': ['Logloss', 'CrossEntropy']
#}
#mod_cb = GridSearchCV(cb.CatBoostClassifier(task_type="GPU"), cv=3, param_grid=cat_boost_params, scoring='average_precision')
#mod_cb.fit(X_train[numerical_features], y_train)
#print('CbGridSearch')
#print('time:', time.time() - st_time)

In [ ]:
#print(mod_cb.best_params_)

In [ ]:
st_time = time.time()
lgbm_params = {
    'max_depth': [2,5,8],
    'n_estimators':[200, 500, 1000],
    'learning_rate': [0.03, 0.05, 0.1],
    'metric': ['binary', 'regression']
}
mod_lgbm = GridSearchCV(lgb.LGBMClassifier(), cv=3, param_grid=lgbm_params, scoring='average_precision')
mod_lgbm.fit(X_train[numerical_features], y_train)
print('CbGridSearch')
print('time:', time.time() - st_time)

In [ ]:
print(mod_lgbm.best_params_)

__Задание 4. (3.5 балла)__

Добавьте категориальные признаки к вещественным следующими способами:

* как OHE признаки;
* как счетчики со сглаживанием.

При подсчете счетчиков запрещается использование циклов. 

На получившихся датасетах подберите параметры у каждого из алгоритмов. Как меняется время, необходимое для обучения модели в зависимости от способа кодирования? Сравните полученные результаты с встроенными методами обработки категориальных признаков. 

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

__Задание 5. (1 балл)__

Реализуйте блендинг подобранных в предыдущем задании моделей и сравните качество.

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

__Задание 6. (1.5 балла)__

В задании 3 вы подобрали гиперпараметры для LightGBM и CatBoost на вещественных признаках. Визуализируйте важности признаков, посчитанные этими алгоритмами, в виде горизонтального bar-plot (отсортируйте признаки по убыванию важности, подпишите названия признаков по оси y).

Для каждого из двух алгоритмов удалите неважные признаки (обычно по bar-plot хорошо видно порог на важность, с которого начинается хвост неважных признаков) и обучите ту же модель на получившихся данных. Сильно ли упало качество при удалении признаков, которые модель считает неважными?

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ